In [1]:
import paho.mqtt.client as mqtt
import paho.mqtt.publish as publish

from pymongo import MongoClient
from bson.objectid import ObjectId

import time,datetime
import os 
import schedule

import pandas as pd
import numpy as np

import math

import matplotlib.pyplot as plt
from matplotlib import style

from copy import deepcopy
from IPython.display import clear_output

In [2]:
def realtimetotalstep():
    print('realtimetotalstep')
    conn = MongoClient('120.126.136.17')
    db = conn.Tracker
    collection = [db.User_Info]# db.db2, db.dn2, db.james, db.leo
    userInfo = []
    for col in collection:
        cursor = col.find({})
        df_user = pd.DataFrame(list(cursor))
    collection = db.realtimetotalsteps
    collection.delete_many({})
    for username in df_user['username']:
        if username == 'pitest' or username == 'jack' or username == 'manager':
            continue
        try: 
            collection = [db[username]]
            clean_data = []
            end = datetime.datetime.now().timestamp()
            start = (datetime.datetime.now().replace(hour=0,minute=0,second=0,microsecond=0)).timestamp()
            for col in collection:
                cursor = col.find({'timestamp': {'$gte': start, '$lt': end}})
                df = pd.DataFrame(list(cursor))
            df.replace('', np.nan, inplace=True)
            df.fillna(method='ffill', inplace=True)
            if 'hr_value' in df:
                df['hr_value'] = df['hr_value'].astype(float)
            if 'o2_value' in df:
                df['o2_value'] = df['o2_value'].astype(float)
            if 'latitude' in df:
                df['latitude'] = df['latitude'].astype(float)
            if 'longitude' in df:
                df['longitude'] = df['longitude'].astype(float)
            if 'step_value' in df:
                df['step_value'] = df['step_value'].astype(float)
            if 'hr_value' in df:    
                df = df[(df['hr_value'] != 0)]
            if 'step_value'  in df:
                totalstep = df['step_value'].sum()
                hourtotalstep = df.loc[df['hour'] == datetime.datetime.now().hour,['step_value']].values.sum()
            else:
                totalstep = 0
                hourtotalstep = 0
            now = datetime.datetime.now()
            dateStr = now.strftime('%Y-%m-%d-%H-%M')
            lis = dateStr.split('-')
            lis = list(map(int, lis))
            collection = db.realtimetotalsteps# db.db2, db.dn2, db.james, db.leo
            mydict = { 'year':lis[0], 'month':lis[1], 'day':lis[2], 'hour':lis[3], 'minute':lis[4],
                     'totalsteps':totalstep,'hourtotalstep':hourtotalstep,'user':username}

            collection.insert_one(mydict)
        except Exception as e:
            print(e)
            pass
        conn.close()

In [3]:
def getRankData():
    print('getRankData')
    t = time.localtime(time.time())
    year = t.tm_year
    month = t.tm_mon
    day = t.tm_mday
    hour = t.tm_hour
    rlist = []

    client=mqtt.Client()
    client.connect("120.126.136.17",1883,60)
    topic = ""
    mongo = MongoClient('120.126.136.17', 27017)
    db = mongo['Tracker']


    collection = db['User_Info']
    friendCol = collection.find()

    for x in friendCol:
        name = x['username']
        if name!= 'pitest' and name!= 'jack' and name!= 'manager':
            rlist.append(name)
    rsize=len(rlist)
    
    collection = db['rank']
    
    data = collection.find({'year':year,'month':month,'day':day,'hour':hour})

    #排名
    for x in data:
        topic = x['user']
#         print(topic)
        rank =1
        pre=99999
        pro=-99999
        for name in rlist:
            if topic != name:
                if x[name]>0 and x[name]<pre:
                    rank=rank+1
                    pre=x[name]                 
                if x[name]<0 and x[name]>pro:
                    pro=x[name]
        if rank==1 and pre==99999:
            rank_mes_top = str(rank)+"/"+str(rsize)
            rank_mes_top = '%-7s' % rank_mes_top
            rank_mes_top = "排名 "+ rank_mes_top
            rank_mes_bot = "第"+str(rank)+"名,好棒棒"
            rank_mes=rank_mes_top+rank_mes_bot
#             print(rank_mes)
            client.publish(topic,rank_mes) 
        elif pro==-99999:
            rank_mes_top = str(rank)+"/"+str(rsize)
            rank_mes_top = '%-7s' % rank_mes_top
            rank_mes_top = "排名 "+ rank_mes_top
            rank_mes_bot = "第"+str(rank)+"名,太混囉"
            rank_mes=rank_mes_top+rank_mes_bot
#             print(rank_mes)
            client.publish(topic,rank_mes)
        else:
            rank_mes_top = str(rank)+"/"+str(rsize)
            rank_mes_top = '%-7s' % rank_mes_top
            rank_mes_top = "排名 "+ rank_mes_top
            rank_mes_bot = "第"+str(rank)+"名,再加油"
            rank_mes=rank_mes_top+rank_mes_bot
#             print(rank_mes)
            client.publish(topic,rank_mes)
        time.sleep(0.01)
            
    time.sleep(5)
#     print("======================")
    
    #前一名
    data = collection.find({'year':year,'month':month,'day':day,'hour':hour})
    
    for x in data:
        topic = x['user']
#         print(topic)
        rank =1
        pre=99999
        pro=-99999
        for name in rlist:
            if topic != name:
                if x[name]>0 and x[name]<pre:
                    rank=rank+1
                    pre=x[name]                 
                if x[name]<0 and x[name]>pro:
                    pro=x[name]
        if rank==1 and pre==99999:
            rank_mes = "保持冠軍喔!"
#             print(rank_mes)
            client.publish(topic,rank_mes) 
        else:
            rank_mes_top = "與前一名落差"
            rank_mes_bot = str(int(pre))+"步"
            rank_mes=rank_mes_top+rank_mes_bot
#             print(rank_mes)
            client.publish(topic,rank_mes)
            
    time.sleep(5)
#     print("======================")
    
    #後一名
    data = collection.find({'year':year,'month':month,'day':day,'hour':hour})
    
    for x in data:
        topic = x['user']
#         print(topic)
        rank =1
        pre=99999
        pro=-99999
        for name in rlist:
            if topic != name:
                if x[name]>0 and x[name]<pre:
                    rank=rank+1
                    pre=x[name]                 
                if x[name]<0 and x[name]>pro:
                    pro=x[name]
        if pro==-99999:
            rank_mes = "吊車尾加把勁"
#             print(rank_mes)
            client.publish(topic,rank_mes) 
        else:
            rank_mes_top = "與後一名落差"
            rank_mes_bot = str(abs(int(pro)))+"步"
            rank_mes=rank_mes_top+rank_mes_bot
#             print(rank_mes)
            client.publish(topic,rank_mes)
            
    mongo.close()
    client.disconnect()

In [4]:
def getHourData():
    print('getHourData')
    t = time.localtime(time.time())
    year = t.tm_year
    month = t.tm_mon
    day = t.tm_mday
    hour = t.tm_hour

    client=mqtt.Client()
    client.connect("120.126.136.17",1883,60)
    topic = ""
    

    mongo = MongoClient('120.126.136.17', 27017)
    db = mongo['Tracker']
    collection = db['prediction']
    
    data = collection.find({'year':year,'month':month,'day':day,'hour':hour,'type':'hourly'})
    
    for x in data:
        topic = x['user']
#         print(topic)
        
        stepCol = db['realtimetotalsteps']
        step = stepCol.find_one({'user':topic})
        top = '步數:%5s步' % int(step['totalsteps'])
        bot = '目標:%5s步' % x['prediction']
        mes = top+bot
#         print(mes)
        client.publish(topic,mes)
        time.sleep(0.01)
    
    mongo.close()
    client.disconnect()

In [5]:
dfsize = -1

In [6]:
def insert_rank_relationship():
    print('insert_rank_relationship')
    conn = MongoClient('120.126.136.17')
    db = conn.Tracker
    collection = [db.realtimetotalsteps]# db.db2, db.dn2, db.james, db.leo
    server = "120.126.136.17"
    port = 1883 #port
    
    for col in collection:
        cursor = col.find({})
        total = pd.DataFrame(list(cursor))
    timedf = total.loc[(total['month'] == datetime.datetime.now().month) 
                            & (total['day']== datetime.datetime.now().day) 
                            & (total['hour'] == datetime.datetime.now().hour)].reset_index()
    timedf.drop(['index','_id'], inplace=True, axis=1)
    timedf = timedf.set_index('user')
    # remove duplicate index
    timedf = timedf.loc[~timedf.index.duplicated(keep='last')]
    
    global dfsize
    if timedf.shape[0] < dfsize:
        time.sleep(3)
        # try one more time
        for col in collection:
            cursor = col.find({})
            total = pd.DataFrame(list(cursor))
        timedf = total.loc[(total['month'] == datetime.datetime.now().month) 
                                & (total['day']== datetime.datetime.now().day) 
                                & (total['hour'] == datetime.datetime.now().hour)].reset_index()
        timedf.drop(['index','_id'], inplace=True, axis=1)
        timedf = timedf.set_index('user')
        # remove duplicate index
        timedf = timedf.loc[~timedf.index.duplicated(keep='last')]
    dfsize = timedf.shape[0]
    try :
        for i in timedf.index:
            timedf[i] = 0
        for index, row in timedf.iterrows():
            for i in timedf.index:
                timedf.loc[index,i] = timedf.loc[i,'totalsteps'] - timedf.loc[index,'totalsteps']
        timedf.drop(['totalsteps', 'hourtotalstep'], inplace=True, axis=1)
        timedf.reset_index(inplace=True)
#         print(timedf)
        for index, rows in timedf.iterrows():
    #         print(rows[0])
            for i in range(5,len(rows)):
    #             print(timedf.columns[i],rows[i])
                if (int(rows[i]) <= 50) and (int(rows[i]) >= 20):
                    mqttc = mqtt.Client()
                    mqttc.connect(server, port) 
                    mqttc.publish(rows[0], "快超越,     " + "差" + str(int(rows[i])) +"步")   
                    time.sleep(5)
                    getrankData()
                elif (int(rows[i]) >= -50) and (int(rows[i]) <= -20):
                    mqttc = mqtt.Client()
                    mqttc.connect(server, port)
                    mqttc.publish(rows[0], "快被超越,   " + "差" + str(abs(int(rows[i]))) +"步")
                    time.sleep(5)
                    getrankData()
        records = timedf.to_dict('records')
        if records != []:
            collection = db.rank
            collection.delete_many({})
            collection.insert_many(records)  
        else:
            print('records', records)
    except Exception as e:
        print(e)
    conn.close()

In [7]:
def lazyreminder(user = 'james'):
    print('lazyreminder')
    conn = MongoClient('120.126.136.17')
    db = conn.Tracker 
    server = "120.126.136.17"
    port = 1883 #port
    
    collection = [db.prediction]
    userInfo = []
    for col in collection:
        cursor = col.find({})
        dfprediction = pd.DataFrame(list(cursor))
    dfprediction = dfprediction.loc[(dfprediction['year'] == datetime.datetime.now().year) & 
                                    (dfprediction['month'] == datetime.datetime.now().month) &
                                    (dfprediction['hour'] == datetime.datetime.now().hour) &
                                    (dfprediction['day'] == datetime.datetime.now().day) &
                                    (dfprediction['user'] == user)]
    #make sure there is only one 
    if dfprediction.shape[0] != 1:
        print('There is something wrong')
        dfprediction = dfprediction[0:1]
    print(dfprediction)
    dfrealtimetotalsteps = pd.DataFrame({})
    while True:
        print('here')
    #     print(dfrealtimetotalsteps)
    #     print(num,'there is something wrong',dfprediction.shape, dfrealtimetotalsteps.shape)
        time.sleep(3)
        collection = [db.realtimetotalsteps]# db.db2, db.dn2, db.james, db.leo
        userInfo = []
        for col in collection:
            cursor = col.find({})
            dfrealtimetotalsteps = pd.DataFrame(list(cursor))
        if (dfrealtimetotalsteps.loc[dfrealtimetotalsteps['user'] == user].all().any()) == True:
            break
    print(dfrealtimetotalsteps.loc[dfrealtimetotalsteps['user'] == user])
    
#     if dfprediction.shape[0] > dfrealtimetotalsteps.shape[0]:
#         dfprediction = dfprediction.iloc[:dfrealtimetotalsteps.shape[0]]
#         print(num,'there is something wrong',dfprediction.shape, dfrealtimetotalsteps.shape)
#         break
#     print(dfprediction['prediction'])
    try:
        
        result = (dfprediction['prediction'] * datetime.datetime.now().minute / 60).values < \
                        (dfrealtimetotalsteps['hourtotalstep'].values * datetime.datetime.now().minute / 60)
        diff = (dfprediction['prediction'] * datetime.datetime.now().minute / 60).values \
                    - (dfrealtimetotalsteps['hourtotalstep'].values * datetime.datetime.now().minute / 60)
        conn.close()
        lis = list(zip(dfprediction['user'].values,result,diff))
        for name,result,diff in lis:
            if result == False:
                mqttc = mqtt.Client()
                mqttc.connect(server, port)
                if math.isnan(diff):
                    diff = 0
                if diff > 5:
                    mqttc.publish('standup' + user, "你在偷懶歐! 還差" + str(int(diff)) +"步")
    except Exception:
        print(str(Exception))
        pass

In [8]:
userdic = {}

def getStep(user,step,s,n):
    stepSeriel = s
    nonzero = n

    if step == 0:
        stepSeriel = stepSeriel+1
        nonzero = 0
        if stepSeriel == 18:
            print("Call Function")
            lazyreminder(user)
            stepSeriel = 0
    elif step > 10:
        nonzero = nonzero+1
        if nonzero == 2:
            stepSeriel = 0
    array = [stepSeriel,nonzero]
    print(array)
    return array

def on_connect(client,userdata,flags,rc):
    print("Connected with result code"+str(rc))
    client.subscribe("test")

def on_message(client,userdata,msg):
    #print(str(msg.payload))
    message = str(msg.payload)
    message = message[2:]
    message = message.split(":")
    try:
        step = int(message[5])
        user = message[0]
    except Exception as e:
        raise
    print(user)
    print(step)

    if user in userdic:
        stepSeriel = userdic[user][0]
        nonzero = userdic[user][1]
        userdic[user] = getStep(user,step,stepSeriel,nonzero)
    else:
        userdic[user] = [0,0]

In [ ]:
client=mqtt.Client()
client.on_connect=on_connect
client.on_message=on_message
client.connect("120.126.136.17",1883,60)
client.loop_forever()

Connected with result code0
demo3
0
demo2
0
demo1
0
demo3
0
[1, 0]
demo2
0
[1, 0]
demo1
0
[1, 0]
demo3
0
[2, 0]
demo2
0
[2, 0]
demo2
0
[3, 0]
demo1
0
[2, 0]
demo3
0
[3, 0]
demo2
0
[4, 0]
demo1
0
[3, 0]
demo3
0
[4, 0]
demo2
0
[5, 0]
demo1
0
[4, 0]
demo2
0
[6, 0]
demo3
0
[5, 0]
demo1
0
[5, 0]
demo3
0
[6, 0]
demo2
0
[7, 0]
demo3
0
[7, 0]
demo1
0
[6, 0]
demo2
0
[8, 0]
demo3
0
[8, 0]
demo1
0
[7, 0]
demo2
0
[9, 0]
demo3
0
[9, 0]
demo1
0
[8, 0]
demo2
0
[10, 0]
demo3
0
[10, 0]
demo1
0
[9, 0]
demo2
0
[11, 0]
demo3
0
[11, 0]
demo2
0
[12, 0]
demo1
0
[10, 0]
demo2
0
[13, 0]
demo3
0
[12, 0]
demo1
0
[11, 0]
demo2
0
[14, 0]
demo3
0
[13, 0]
demo1
0
[12, 0]
demo2
0
[15, 0]
demo3
0
[14, 0]
demo3
0
[15, 0]
demo1
0
[13, 0]
demo3
0
[16, 0]
demo2
0
[16, 0]
demo1
0
[14, 0]
demo2
0
[17, 0]
demo3
0
[17, 0]
demo1
0
[15, 0]
demo2
0
Call Function
lazyreminder
                           _id  day  hour  month  prediction  totalsteps  \
2933  5cee98ffcd71ec0eaca655a7   31  11.0      5          57      4770.0   

   

demo3
0
[9, 0]
demo1
0
[15, 0]
demo2
0
[1, 0]
demo1
0
[16, 0]
demo3
0
[10, 0]
demo3
0
[11, 0]
demo2
0
[2, 0]
demo1
0
[17, 0]
demo2
0
[3, 0]
demo1
0
Call Function
lazyreminder
                           _id  day  hour  month  prediction  totalsteps  \
2909  5cee98f8cd71ec0eaca6558f   31  11.0      5          57      4770.0   

        type   user  year  
2909  hourly  demo1  2019  
here
                        _id  day  hour  hourtotalstep  minute  month  \
5  5cf09d8ec4e5a665b1e52296   31    11           28.0      20      5   

   totalsteps   user  year  
5        28.0  demo1  2019  
[0, 0]
demo1
0
[1, 0]
demo2
0
[4, 0]
demo2
0
[5, 0]
demo1
0
[2, 0]
demo2
0
[6, 0]
demo1
0
[3, 0]
demo2
0
[7, 0]
demo1
0
[4, 0]
demo3
0
[12, 0]
demo2
0
[8, 0]
demo1
0
[5, 0]
demo2
0
[9, 0]
demo1
0
[6, 0]
demo2
0
[10, 0]
demo2
0
[11, 0]
demo3
0
[13, 0]
demo1
0
[7, 0]
demo2
0
[12, 0]
demo3
0
[14, 0]
demo1
0
[8, 0]
demo2
0
[13, 0]
demo3
0
[15, 0]
demo1
0
[9, 0]
demo2
0
[14, 0]
demo3
0
[16, 0]
demo3
0
[17, 0]


                        _id  day  hour  hourtotalstep  minute  month  \
6  5cf09ffdc4e5a665b1e5230f   31    11           46.0      31      5   

   totalsteps   user  year  
6        46.0  demo2  2019  
[0, 0]
demo4
0
[1, 0]
demo1
0
[1, 0]
demo2
0
[1, 0]
demo2
0
[2, 0]
demo1
0
[2, 0]
demo1
0
[3, 0]
demo1
0
[4, 0]
demo2
0
[3, 0]
demo3
0
[8, 0]
demo1
0
[5, 0]
demo2
0
[4, 0]
demo1
0
[6, 0]
demo4
0
[2, 0]
demo2
0
[5, 0]
demo1
0
[7, 0]
demo1
0
[8, 0]
demo4
0
[3, 0]
demo1
0
[9, 0]
demo2
0
[6, 0]
demo1
0
[10, 0]
demo2
0
[7, 0]
demo2
0
[8, 0]
demo1
0
[11, 0]
demo1
0
[12, 0]
demo2
0
[9, 0]
demo4
0
[4, 0]
demo1
0
[13, 0]
demo2
0
[10, 0]
demo1
0
[14, 0]
demo4
0
[5, 0]
demo1
0
[15, 0]
demo2
22
[10, 1]
demo1
0
[16, 0]
demo3
0
[9, 0]
demo2
0
[11, 0]
demo2
0
[12, 0]
demo2
0
[13, 0]
demo1
0
[17, 0]
demo1
0
Call Function
lazyreminder
                           _id  day  hour  month  prediction  totalsteps  \
2909  5cee98f8cd71ec0eaca6558f   31  11.0      5          57      4770.0   

        type   use

demo2
0
[4, 0]
demo3
0
[14, 0]
demo1
0
[10, 0]
demo2
0
[5, 0]
demo3
0
[15, 0]
demo1
0
[11, 0]
demo2
0
[6, 0]
demo3
0
[16, 0]
demo1
0
[12, 0]
demo2
0
[7, 0]
demo3
0
[17, 0]
demo1
0
[13, 0]
demo2
0
[8, 0]
demo2
0
[9, 0]
demo3
0
Call Function
lazyreminder
                           _id  day  hour  month  prediction  totalsteps  \
2957  5cee9903cd71ec0eaca655bf   31  11.0      5          57      4770.0   

        type   user  year  
2957  hourly  demo3  2019  
here
                        _id  day  hour  hourtotalstep  minute  month  \
7  5cf0a5a1c4e5a665b1e524fa   31    11          536.0      55      5   

   totalsteps   user  year  
7       536.0  demo3  2019  
[0, 0]
demo1
0
[14, 0]
demo2
0
[10, 0]
demo3
0
[1, 0]
demo1
0
[15, 0]
demo2
0
[11, 0]
demo3
0
[2, 0]
demo1
0
[16, 0]
demo1
0
[17, 0]
demo2
0
[12, 0]
demo3
0
[3, 0]
demo1
0
Call Function
lazyreminder
                           _id  day  hour  month  prediction  totalsteps  \
2909  5cee98f8cd71ec0eaca6558f   31  11.0      5       

                        _id  day  hour  hourtotalstep  minute  month  \
5  5cf0a91ec4e5a665b1e52694   31    12           44.0      10      5   

   totalsteps   user  year  
5       278.0  demo1  2019  
[0, 0]
demo3
0
[1, 0]
demo1
0
[1, 0]
demo3
0
[2, 0]
demo1
0
[2, 0]
demo1
0
[3, 0]
demo3
0
[3, 0]
demo1
0
[4, 0]
demo3
0
[4, 0]
demo1
0
[5, 0]
demo3
0
[5, 0]
demo1
0
[6, 0]
demo3
0
[6, 0]
demo1
0
[7, 0]
demo3
0
[7, 0]
demo1
0
[8, 0]
demo3
0
[8, 0]
demo1
0
[9, 0]
demo1
0
[10, 0]
demo3
0
[9, 0]
demo1
0
[11, 0]
demo3
0
[10, 0]
demo1
0
[12, 0]
demo3
0
[11, 0]
demo1
0
[13, 0]
demo3
0
[12, 0]
demo1
0
[14, 0]
demo3
0
[13, 0]
demo1
0
[15, 0]
demo1
0
[16, 0]
demo3
0
[14, 0]
demo1
0
[17, 0]
demo3
0
[15, 0]
demo1
0
Call Function
lazyreminder
                           _id  day  hour  month  prediction  totalsteps  \
2910  5cee98f8cd71ec0eaca65590   31  12.0      5         722      5026.0   

        type   user  year  
2910  hourly  demo1  2019  
here
                        _id  day  hour  hourtot

...

demo3
0
[9, 0]
demo1
0
[2, 0]
demo4
0
[10, 0]
demo2
0
[14, 0]
demo3
0
[10, 0]
demo1
0
[3, 0]
demo4
0
[11, 0]
demo2
0
[15, 0]
demo3
0
[11, 0]
demo1
0
[4, 0]
demo4
0
[12, 0]
demo2
0
[16, 0]
demo3
0
[12, 0]
demo1
0
[5, 0]
demo2
0
[17, 0]
demo3
0
[13, 0]
demo1
0
[6, 0]
demo2
0
Call Function
lazyreminder
                           _id  day  hour  month  prediction  totalsteps  \
2938  5cee98ffcd71ec0eaca655ac   31  16.0      5         270      7752.0   

        type   user  year  
2938  hourly  demo2  2019  
here
                        _id  day  hour  hourtotalstep  minute  month  \
6  5cf0e26cc4e5a665b1e541b0   31    16           25.0      14      5   

   totalsteps   user  year  
6       296.0  demo2  2019  
[0, 0]
demo3
0
[14, 0]
demo1
0
[7, 0]
demo2
0
[1, 0]
demo3
0
[15, 0]
demo1
0
[8, 0]
demo2
0
[2, 0]
demo3
0
[16, 0]
demo1
0
[9, 0]
demo2
0
[3, 0]
demo3
0
[17, 0]
demo1
0
[10, 0]
demo2
0
[4, 0]
demo3
0
Call Function
lazyreminder
                           _id  day  hour  month  predi

demo3
0
[7, 0]
demo1
0
[7, 0]
demo2
0
[6, 0]
demo3
0
[8, 0]
demo1
0
[8, 0]
demo2
0
[7, 0]
demo3
0
[9, 0]
demo1
0
[9, 0]
demo1
0
[10, 0]
demo2
0
[8, 0]
demo3
0
[10, 0]
demo1
0
[11, 0]
demo2
0
[9, 0]
demo3
0
[11, 0]
demo1
0
[12, 0]
demo2
0
[10, 0]
demo3
0
[12, 0]
demo1
0
[13, 0]
demo2
0
[11, 0]
demo3
0
[13, 0]
demo1
0
[14, 0]
demo2
0
[12, 0]
demo3
0
[14, 0]
demo1
0
[15, 0]
demo2
0
[13, 0]
demo3
0
[15, 0]
demo1
0
[16, 0]
demo2
0
[14, 0]
demo3
0
[16, 0]
demo1
0
[17, 0]
demo1
0
Call Function
lazyreminder
                           _id  day  hour  month  prediction  totalsteps  \
2914  5cee98f8cd71ec0eaca65594   31  16.0      5         270      7752.0   

        type   user  year  
2914  hourly  demo1  2019  
here
                        _id  day  hour  hourtotalstep  minute  month  \
5  5cf0e535c4e5a665b1e542e5   31    16            0.0      26      5   

   totalsteps   user  year  
5       637.0  demo1  2019  
[0, 0]
demo2
0
[15, 0]
demo3
0
[17, 0]
demo1
0
[1, 0]
demo2
0
[16, 0]
demo3
0


demo1
0
[6, 0]
demo2
0
[1, 0]
demo3
0
[3, 0]
demo1
0
[7, 0]
demo2
0
[2, 0]
demo3
0
[4, 0]
demo1
0
[8, 0]
demo2
0
[3, 0]
demo3
0
[5, 0]
demo1
0
[9, 0]
demo2
0
[4, 0]
demo3
0
[6, 0]
demo1
0
[10, 0]
demo2
0
[5, 0]
demo3
0
[7, 0]
demo1
0
[11, 0]
demo2
0
[6, 0]
demo3
0
[8, 0]
demo1
0
[12, 0]
demo2
0
[7, 0]
demo3
0
[9, 0]
demo1
0
[13, 0]
demo2
0
[8, 0]
demo3
0
[10, 0]
demo1
0
[14, 0]
demo2
0
[9, 0]
demo3
0
[11, 0]
demo1
0
[15, 0]
demo2
0
[10, 0]
demo3
0
[12, 0]
demo1
0
[16, 0]
demo2
0
[11, 0]
demo3
0
[13, 0]
demo1
0
[17, 0]
demo2
0
[12, 0]
demo3
0
[14, 0]
demo1
0
Call Function
lazyreminder
                           _id  day  hour  month  prediction  totalsteps  \
2914  5cee98f8cd71ec0eaca65594   31  16.0      5         270      7752.0   

        type   user  year  
2914  hourly  demo1  2019  
here
                        _id  day  hour  hourtotalstep  minute  month  \
5  5cf0e7e7c4e5a665b1e5441b   31    16            0.0      37      5   

   totalsteps   user  year  
5       637.0  demo1 

                        _id  day  hour  hourtotalstep  minute  month  \
6  5cf0eb00c4e5a665b1e5457a   31    16           76.0      51      5   

   totalsteps   user  year  
6       347.0  demo2  2019  
[0, 0]
demo1
0
[6, 0]
demo2
0
[1, 0]
demo1
0
[7, 0]
demo2
0
[2, 0]
demo1
0
[8, 0]
demo2
0
[3, 0]
demo1
0
[9, 0]
demo2
0
[4, 0]
demo1
0
[10, 0]
demo2
0
[5, 0]
demo1
0
[11, 0]
demo2
0
[6, 0]
demo1
0
[12, 0]
demo2
0
[7, 0]
demo1
0
[13, 0]
demo2
0
[8, 0]
demo1
0
[14, 0]
demo2
0
[9, 0]
demo1
0
[15, 0]
demo2
0
[10, 0]
demo1
0
[16, 0]
demo2
0
[11, 0]
demo1
0
[17, 0]
demo2
0
[12, 0]
demo1
0
Call Function
lazyreminder
                           _id  day  hour  month  prediction  totalsteps  \
2914  5cee98f8cd71ec0eaca65594   31  16.0      5         270      7752.0   

        type   user  year  
2914  hourly  demo1  2019  
here
                        _id  day  hour  hourtotalstep  minute  month  \
5  5cf0eb70c4e5a665b1e545ab   31    16            0.0      53      5   

   totalsteps   user  yea

demo1
0
[2, 0]
demo2
0
[14, 0]
demo1
0
[3, 0]
demo2
0
[15, 0]
demo1
0
[4, 0]
demo2
0
[16, 0]
demo1
0
[5, 0]
demo2
0
[17, 0]
demo1
0
[6, 0]
demo2
0
Call Function
lazyreminder
                           _id  day  hour  month  prediction  totalsteps  \
2939  5cee98ffcd71ec0eaca655ad   31  17.0      5         429      7752.0   

        type   user  year  
2939  hourly  demo2  2019  
here
                        _id  day  hour  hourtotalstep  minute  month  \
6  5cf0ef31c4e5a665b1e54746   31    17            0.0       9      5   

   totalsteps   user  year  
6       347.0  demo2  2019  
[0, 0]
demo1
0
[7, 0]
demo2
0
[1, 0]
demo1
0
[8, 0]
demo2
0
[2, 0]
demo1
0
[9, 0]
demo2
0
[3, 0]
demo1
0
[10, 0]
demo2
0
[4, 0]
demo1
0
[11, 0]
demo2
0
[5, 0]
demo1
0
[12, 0]
demo2
0
[6, 0]
demo1
0
[13, 0]
demo2
0
[7, 0]
demo1
0
[14, 0]
demo2
0
[8, 0]
demo1
0
[15, 0]
demo2
0
[9, 0]
demo1
0
[16, 0]
demo2
0
[10, 0]
demo1
0
[17, 0]
demo2
0
[11, 0]
demo1
0
Call Function
lazyreminder
                           

In [14]:
realtimetotalstep()
insert_rank_relationship()
lazyreminder()

realtimetotalstep
insert_rank_relationship
lazyreminder
                           _id  day  hour  month  prediction  totalsteps  \
2863  5cee9886cd71ec0eaca65561   30  13.0      5         880      5026.0   
2983  5cee9907cd71ec0eaca655d9   30  13.0      5         880      5026.0   
3007  5cee990fcd71ec0eaca655f1   30  13.0      5         880      5026.0   
3031  5cee9912cd71ec0eaca65609   30  13.0      5         880      5026.0   
3092  5ceeae4dcd71ec0eaca6564a   30  13.0      5         880      5026.0   
3116  5ceeae56cd71ec0eaca65662   30  13.0      5         880      5026.0   
3140  5ceeae5bcd71ec0eaca6567a   30  13.0      5         880      5026.0   
3164  5ceeae60cd71ec0eaca65692   30  13.0      5         880      5026.0   

        type   user  year  
2863  hourly  james  2019  
2983  hourly  demo3  2019  
3007  hourly  demo2  2019  
3031  hourly  demo1  2019  
3092  hourly  shuan  2019  
3116  hourly  james  2019  
3140  hourly    leo  2019  
3164  hourly   jeng  2019  
Empty D

In [33]:
schedule.every(1).minutes.do(realtimetotalstep)
schedule.every(2).minutes.do(insert_rank_relationship)
schedule.every(10).minutes.do(lazyreminder)
schedule.every(1).hour.at(':02').do(getRankData)
schedule.every(1).hour.at(':00').do(getHourData)
# schedule.every(4).minutes.do(getHourData)

Every 1 hour at 00:00:00 do getHourData() (last run: [never], next run: 2019-05-30 00:00:00)

In [ ]:
while True:
    schedule.run_pending()
    time.sleep(1)

In [ ]:
realtimetotalstep()
lazyreminder()

In [12]:
# getHourData()
getRankData()
# realtimetotalstep()

getRankData


In [ ]:
conn = MongoClient('120.126.136.17')
db = conn.Tracker
collection = [db.prediction]# db.db2, db.dn2, db.james, db.leo
user = []

for col in collection:
    cursor = col.find({})
    prediction = pd.DataFrame(list(cursor))
timedf = prediction.loc[(prediction['month'] == datetime.datetime.now().month) 
                        & (prediction['day']== datetime.datetime.now().day) 
                        & (prediction['hour'] == datetime.datetime.now().hour)].reset_index()
timedf.drop(['index','_id'], inplace=True, axis=1)
timedf = timedf.set_index('user')
# remove duplicate index
timedf = timedf.loc[~timedf.index.duplicated(keep='last')]
for i in timedf.index:
    timedf[i] = 0
for index, row in timedf.iterrows():
    for i in timedf.index:
        timedf.loc[index,i] = timedf.loc[i,'totalsteps'] - timedf.loc[index,'totalsteps']
timedf.drop(['prediction', 'totalsteps', 'type'], inplace=True, axis=1)
timedf.reset_index(inplace=True)
records = timedf.to_dict('records')
if records != []:
    collection = db.rank
    collection.insert_many(records)
else:
    print('records', records)
conn.close()